# SLME - Workflows

To calculate the SLME, we require the absorption coefficient of the material, as well as the direct and indirect band gap. As the CuAu and chalcopyrite materials we investigate all have direct band gaps in the $\Gamma$ point, we can obtain a very accurate band gap from the optics calculation itself.

In [1]:
# Notebook header

import os, json
from monty.json import MontyDecoder
from pymatgen import Structure
from custodian.vasp.handlers import VaspErrorHandler
from vscworkflows.config import load_config
from vscworkflows.handlers.core import ElectronicConvergenceMonitor
from vscworkflows.fireworks.core import OptimizeFW, OpticsFW
from vscworkflows.workflows.core import _set_up_functional_params, \
    _set_up_relative_directory, Workflow

First we will define the directory in which we will perform the calculations on the cluster. Note that if you want to run this notebook to replicate my results, you will have to adjust this directory!

In [2]:
cluster = "leibniz"

# Set up the scratch dir
if cluster == "breniac":
    scratch_dir = "/scratch/leuven/202/vsc20248/slme/high-throughput"   # This should be changed to your directory
else:
    scratch_dir = "/scratch/antwerpen/202/vsc20248/slme/high-throughput"   # This should be changed to your directory

Next we'll load the launchpad on which we will store the fireworks. If you haven't configured a launchpad yet, open a terminal and use `vsc config launchpad` to do so. The command below loads the 'base' launchpad (i.e. the default). Change this if necessary.

In [3]:
# Load the base launchpad
lpad = load_config("launchpad", cluster)                # Load whichever launchpad you have configured 

After running the [`structures.ipynb`](structures.ipynb) notebook, you should have all structure files stored in the `structures` directory. The line of code below makes a list of all the `.json` files in this directory.

In [4]:
structure_files = [file for file in os.listdir("structures") if ".json" in file]

To submit the workflows to the mongoDB server specified by the `Launchpad` defined above, we will loop over all of the structure files and generate a workflow using the `get_wf_optics` method. This is done in the cell below.

In [5]:
# Functional
# functional=("pbe", {})
functional=("hse06", {})
number_nodes = 8

scratch_dir = "/scratch/antwerpen/202/vsc20248/slme/high-throughput"

# Set the error handlers
handlers = [
    VaspErrorHandler(errors_subset_to_catch=["zbrent"]),
    ElectronicConvergenceMonitor()
]

# Change default settings, for e.g. parallelization
user_incar_settings = {
    "KPAR": 4,
    "NPAR": 8,
    "ALGO": "Normal",
}

user_kpoints_settings_optimize = {"reciprocal_density": 100}
user_kpoints_settings_optics = {"gamma_density": 500}

wf_list = []

for file in structure_files:
    
    structure = Structure.from_file(os.path.join("structures", file)) 
    directory = os.path.join(scratch_dir, 
                             file.split(".")[0].split("_")[1],
                             file.split(".")[0].split("_")[0])
    
    # Add number of nodes to spec, or "none"
    if number_nodes is not None and number_nodes != 0:
        spec = {"_fworker": str(number_nodes) + "nodes"}
    else:
        spec = {}
    
    # 1. Set up the geometry optimization with PBE
    vasp_input_params = _set_up_functional_params(
        functional=("pbe", {})
    )
    spec.update(
        {"_launch_dir": _set_up_relative_directory(directory, ("pbe", {}), "optimize")}
    )

    # Override the settings with the user specifications
    if user_kpoints_settings_optimize is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optimize
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the Firework
    optimize_fw_pbe = OptimizeFW(structure=structure,
                             vasp_input_params=vasp_input_params,
                             custodian=handlers,
                             spec=spec)

    # 2. Set up the geometry optimization with chosen functional
    vasp_input_params = _set_up_functional_params(functional)
    spec.update(
        {"_launch_dir": _set_up_relative_directory(directory, functional, "optimize")}
    )

    # Override the settings with the user specifications
    if user_kpoints_settings_optimize is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optimize
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the Firework
    optimize_fw = OptimizeFW(parents=optimize_fw_pbe,
                             vasp_input_params=vasp_input_params,
                             custodian=handlers,
                             spec=spec)

    # 3. Set up the optics calculation
    vasp_input_params = _set_up_functional_params(functional)
    spec.update({"_launch_dir": _set_up_relative_directory(directory, functional,
                                                           "optics")})
    
    # Override the settings with the user specifications
    if user_kpoints_settings_optics is not None:
        vasp_input_params["user_kpoints_settings"] = user_kpoints_settings_optics
    if user_incar_settings is not None:
        vasp_input_params["user_incar_settings"].update(user_incar_settings)

    # Set up the geometry optimization Firework
    optics_fw = OpticsFW(
        parents=optimize_fw,
        vasp_input_params=vasp_input_params,
        custodian=handlers,
        spec=spec
    )

    # Set up a clear name for the workflow
    workflow_name = "Optics "
    workflow_name += str(structure.composition.reduced_formula).replace(" ", "")
    workflow_name += " " + str(functional[0])

    # Create the workflow
    wf_list.append(
        Workflow(fireworks=[optimize_fw_pbe, optimize_fw, optics_fw],
                 links_dict={optimize_fw_pbe: [optimize_fw], optimize_fw: [optics_fw]},
                 name=workflow_name)
    )

In [14]:
with open("data/solar.json", "r") as file:
    solar_data = json.loads(file.read(), cls=MontyDecoder)
    
issue_structures = [v["structure"] for v in solar_data.values() 
                    if v["issues"]["solar"] is None]
# issue_structures = [str(s.get_space_group_info()[1]) +  for s in issue_structures]

for i, l in enumerate(wf_list):
    s = Structure.from_dict(l.as_dict()["fws"][0]["spec"]["_tasks"][0]["vasp_input_set"]["structure"])
    print(s.get_space_group_info())
    print(s.composition.iupac_formula)

('I-42d', 122)
Cu2 Al2 S4
('P-4m2', 115)
Cu1 Ga1 Se2
('I-42d', 122)
Ag2 Ga2 Te4
('P-4m2', 115)
Cu1 Al1 S2
('I-42d', 122)
Cu2 Ga2 Se4
('P-4m2', 115)
Ag1 Ga1 Te2
('P-4m2', 115)
Ag1 Ga1 Se2
('I-42d', 122)
Cu2 Ga2 Te4
('I-42d', 122)
Cu2 Ga2 S4
('I-42d', 122)
Ag2 In2 S4
('I-42d', 122)
Ag2 Ga2 Se4
('P-4m2', 115)
Cu1 Ga1 Te2
('P-4m2', 115)
Cu1 Ga1 S2
('P-4m2', 115)
Ag1 In1 S2
('P-4m2', 115)
Cu1 In1 Te2
('I-42d', 122)
Cu2 Al2 Se4
('P-4m2', 115)
Ag1 Al1 Te2
('I-42d', 122)
Ag2 In2 Se4
('P-4m2', 115)
Ag1 Al1 S2
('I-42d', 122)
Cu2 In2 Te4
('P-4m2', 115)
Cu1 Al1 Se2
('I-42d', 122)
Ag2 Al2 Te4
('P-4m2', 115)
Ag1 In1 Se2
('I-42d', 122)
Ag2 Al2 S4
('P-4m2', 115)
Ag1 Ga1 S2
('P-4m2', 115)
Cu1 In1 S2
('I-42d', 122)
Ag2 Al2 Se4
('P-4m2', 115)
Ag1 In1 Te2
('I-42d', 122)
Cu2 In2 Se4
('P-4m2', 115)
Cu1 Al1 Te2
('I-42d', 122)
Ag2 Ga2 S4
('I-42d', 122)
Cu2 In2 S4
('P-4m2', 115)
Ag1 Al1 Se2
('I-42d', 122)
Ag2 In2 Te4
('P-4m2', 115)
Cu1 In1 Se2
('I-42d', 122)
Cu2 Al2 Te4


In [6]:
# lpad.bulk_add_wfs(wf_list)
lpad.add_wf(wf_list[32])

2020-02-07 15:48:24,186 INFO Added a workflow. id_map: {-99: 8, -98: 9, -97: 10}


{-99: 8, -98: 9, -97: 10}